# Intersecting the exponent

- https://adventofcode.com/2023/day/4

The task here is to find the number of winning numbers picked on each card. All you need to do for that is to take the length of the intersection of the two sets of numbers.

The card points value is then an exponent of 2; one lower than the size of the intersection: 1 match is $2 ^ 0 = 1$, 2 matches is $2 ^ 1 = 2$, etc. The only corner case to take care of here is when there are no matches, in which case the points value is 0.


In [1]:
import typing as t
from dataclasses import dataclass


@dataclass
class Card:
    id: int
    winning_numbers: frozenset[int]
    my_numbers: frozenset[int]

    @classmethod
    def from_line(cls, line: str) -> t.Self:
        identifier, _, numbers = line.partition(":")
        id_ = int(identifier.partition(" ")[-1])
        winning, _, my = numbers.partition("|")
        return cls(
            id_, frozenset(map(int, winning.split())), frozenset(map(int, my.split()))
        )

    @property
    def matches(self):
        return len(self.winning_numbers & self.my_numbers)

    @property
    def points(self):
        count = self.matches
        return 2 ** (count - 1) if count else 0


test_cards = [
    Card.from_line(line)
    for line in """\
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
""".splitlines()
]

assert [c.points for c in test_cards] == [8, 2, 2, 1, 0, 0]

In [2]:
import aocd

cards = [Card.from_line(line) for line in aocd.get_data(day=4, year=2023).splitlines()]
print("Part 1:", sum(c.points for c in cards))

Part 1: 24542


# Counting cards for Teh Win

For part 2, there is a lot of talk about copying cards in the text. There is no need to copy anything, all we need to do is _count_ the number of cards so far, and use those counts to increase the counts for a number of cards that follow the current card in the list based on the number of matches for the current card.

Keep a list of all the card counts, one value per card, and start at 1. Then, as you go down the list of cards, take the number of matches for that card, add the current count for that card to the right number of cards below it in the list. You don't even have to keep track of the current index in the list as you go because my `Card` class has an `id` attribute that's equal to the index plus 1.

For the sample list of cards, the counts for the six cards start at `[1, 1, 1, 1, 1, 1]`. Then process each card; in the following list of steps the current card count is underscored and the card counts that are affected are shown in italics:

1. Card 1 has one copy and 4 matches, so add 1 to the 4 next cards: <code>[<u>1</u>, <i>2, 2, 2, 2</i>, 1]</code>
2. Card 2 has 2 copies now and 2 matches, so add 2 to the next 2 cards: <code>[1, <u>2</u>, <i>4, 4</i>, 2, 1]</code>
3. Card 3 has 4 copies now, and has 2 matches, so add 4 to the next 2 cards: <code>[1, 2, <u>4</u>, <i>8, 6</i>, 1]</code>
4. Card 4 has 8 copies now, and has 1 match, so add 8 to the next card in the list: <code>[1, 2, 4, <u>8</u>, <i>14</i>, 9]</code>
5. Card 5 has 14 copies, has no matches so the list of counts is unchanged: <code>[1, 2, 4, 8, <u>14</u>, 9]</code>
6. It doesn't matter how many matches card 6 has, there are no cards that follow it.

Then simply sum the list of card counts for the total. Just make sure you don't try to add counts past the end of the list.

Alternatively, you could keep a running total and pop off values from the list as you go, but the number of cards in the input is small enough that the additional complexity is not going to be worth it.


In [3]:
def cards_won(cards: t.Sequence[Card]) -> int:
    counts = [1] * len(cards)
    for card in cards:
        count = counts[card.id - 1]
        for idx in range(card.id, card.id + card.matches):
            if idx == len(cards):
                break
            counts[idx] += count
    return sum(counts)


assert cards_won(test_cards) == 30

In [4]:
print("Part 2:", cards_won(cards))

Part 2: 8736438
